# A test for fast storage

### Introduction

All we need is contained in the `openpathsampling` package

In [1]:
import openpathsampling as paths

Open the test storage for demonstrataion

In [2]:
storage = paths.storage.Storage('toy_tis.nc', mode='r')

extract an orderparameter and choose an ensemble

In [5]:
op = storage.cv[0]
ens = storage.ensemble[0]

get all values for orderparameters in all samples found in samplesets for a specific ensemble

In [6]:
%%time
result = storage.query.trajectory_orderparameter(op,ens)

CPU times: user 455 ms, sys: 75.5 ms, total: 530 ms
Wall time: 522 ms


In [7]:
%%time
result2 = [op(sample.trajectory) for sset in storage.sampleset for sample in sset if sample.ensemble is ens]

CPU times: user 1.13 s, sys: 186 ms, total: 1.32 s
Wall time: 1.23 s


In [8]:
op_line = paths.OP_Function(name="xval2", fcn=lambda snap : snap.coordinates[0,0])

In [14]:
result2[0][2]

-0.28195512294769287

In [18]:
print op_line(storage.trajectory[0])[2]

-0.281955 dimensionless


the second is slower when run first since it creates the sample objects. Every second time it is much faster while the first implementation has always the same speed. Also the second approach is much more flexible and is a one-liner compared to the 20+ line implementation of the first call.

In [19]:
%%time
result = storage.query.trajectory_orderparameter(op,ens)

CPU times: user 446 ms, sys: 72.3 ms, total: 518 ms
Wall time: 544 ms


In [20]:
%%time
result2 = [op(sample.trajectory) for sset in storage.sampleset for sample in sset if sample.ensemble is ens]

CPU times: user 9.15 ms, sys: 4.9 ms, total: 14.1 ms
Wall time: 11.8 ms


compare all element

In [22]:
assert(len([x for x in [res[0] == res[1] for res in zip(result, result2)] if x if False]) == 0)

lets check the same for the replica ID

In [23]:
%%time
result_rep1 = storage.query.trajectory_orderparameter(op, replica=5)

CPU times: user 451 ms, sys: 72.3 ms, total: 523 ms
Wall time: 517 ms


In [24]:
%%time
result_rep2 = [op(sample.trajectory) for sset in storage.sampleset for sample in sset if sample.replica == 5]

CPU times: user 14 ms, sys: 4.36 ms, total: 18.4 ms
Wall time: 15.6 ms


In [25]:
print result[7][2]
print result2[7][2]

-0.281955122948
-0.281955122948


let also get a list of trajectory lengths

In [26]:
%%time
result_len1 = storage.query.trajectory_length(replica=5)

CPU times: user 449 ms, sys: 71.9 ms, total: 521 ms
Wall time: 515 ms


In [27]:
%%time
result_len2 = [len(sample.trajectory) for sset in storage.sampleset for sample in sset if sample.replica == 5 ]

CPU times: user 1.18 ms, sys: 561 µs, total: 1.74 ms
Wall time: 1.35 ms


In [28]:
print result_len1
print result_len2

[51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50, 50, 50, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 65, 65, 65, 65, 65, 65, 65]
[51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50, 50, 50, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 65, 65, 65, 65, 65, 65, 65]


In [29]:
mp = storage.movepath[2]
filter(lambda x : x is not None, mp.traverse_dfs(lambda x : x.mover if hasattr(x, 'mover') else None))